## 나의 사진으로 나만의 이야기 만들기


## 기본 세팅

In [ ]:
!pip install httpx==0.27.2
!pip install openai
!pip install gradio
!pip install fpdf
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=8710

In [ ]:
# import lines
import requests
import zipfile
import io
import os
import base64
import matplotlib.pyplot as plt
import time
import gradio as gr
import asyncio
import threading
from io import BytesIO
from openai import OpenAI
from PIL import Image
from IPython.display import display, HTML
from fpdf import FPDF  # PDF 생성 라이브러리
import docx  # Word 파일 생성 라이브러리
import webbrowser  # 웹 브라우저로 링크 열기 위해 사용

In [ ]:
# 파일 ID와 다운로드 URL 리스트
sample_image_url1 = "https://drive.google.com/uc?id=1vuhPvNwlYqB4eRl0nvEPwcRoLnxy0mhx"
sample_image_url2 = "https://drive.google.com/uc?id=1tvV7pVSdi9HT7TT9rQgptqSr6g7cosJz"
sample_image_url3 = "https://drive.google.com/uc?id=1gWJS9PNEHT3R-eFmMhPlaOuBBZXbW0vA"
sample_image_url4 = "https://drive.google.com/uc?id=1ByigXqQ_JPRtcAeBlnSnl32iQUb-A3XQ"
sample_image_url5 = "https://drive.google.com/uc?id=1F2bpo8Z2sTqOyKPlcbBhuM-RRR69Lb6f"
sample_image_url6 = "https://drive.google.com/uc?id=1vyBIxeUbMPQ0FNe9GnC0jMlVZGsv3Tt_"
sample_image_url7 = "https://drive.google.com/uc?id=1A_-hUey9oYNj-eOga63sBO14KrsrkT2u"

image_lst = [
    sample_image_url1,
    sample_image_url2,
    sample_image_url3,
    sample_image_url4,
    sample_image_url5,
    sample_image_url6,
    sample_image_url7,
    ]

# 파일 ID와 다운로드 URL 리스트
file_info = {
    "17kzFOeDkZy-SUQmsIISTLQ7MHZjf0ThT": "sample1.zip", # 샘플파일1
    "1VEaM3Ltz5ZS87EbeFsgkDC6aM0s2iivy": "sample2", # 샘플파일2 건국대
    "1jeib4BD2aG38F0CPi053A1cTMjrgK4Aa": "page1_select_genre.zip", # 1p 장르선택 이미지파일
    "1K4FwYcpc-UBvkStpaUsW0j4clVAomzc6": "page2_select_setting.zip", # 2p 세팅선택 이미지파일
    "16oiuQogYqvquhRkqc44hQpIbVaXVh1uj": "page3_select_drawing_style.zip", # 3p 그림체선택 이미지파일
    "1tBeoFYyG-YQCFfLeuzKOUCrsPRsD7r7R": "home.zip" # 프로젝트 소개
}

base_url = "https://drive.google.com/uc?id="

# 압축 해제할 디렉토리
extract_root = "extracted_files"

# 디렉토리가 없으면 생성
if not os.path.exists(extract_root):
    os.makedirs(extract_root)

# 각 파일 다운로드 및 압축 해제
for file_id, file_name in file_info.items():
    url = f"{base_url}{file_id}"
    print(f"파일 다운로드 중... (ID: {file_id}, 이름: {file_name})")
    response = requests.get(url)

    if response.status_code == 200:
        # 파일 이름을 기준으로 디렉토리 생성
        folder_name = os.path.splitext(file_name)[0]  # 확장자 제거한 폴더 이름
        extract_path = os.path.join(extract_root, folder_name)

        if not os.path.exists(extract_path):
            os.makedirs(extract_path)

        print(f"파일 압축 해제 중... (파일 이름: {file_name})")
        try:
            with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
                zip_ref.extractall(extract_path)
            print(f"파일이 '{extract_path}' 디렉토리에 성공적으로 압축 해제되었습니다. (파일 이름: {file_name})")
        except zipfile.BadZipFile:
            print(f"파일이 압축 파일이 아닙니다. (파일 이름: {file_name})")
    else:
        print(f"파일 다운로드에 실패했습니다. (ID: {file_id}, 이름: {file_name})")

파일 다운로드 중... (ID: 17kzFOeDkZy-SUQmsIISTLQ7MHZjf0ThT, 이름: sample1.zip)
파일 압축 해제 중... (파일 이름: sample1.zip)
파일이 'extracted_files/sample1' 디렉토리에 성공적으로 압축 해제되었습니다. (파일 이름: sample1.zip)
파일 다운로드 중... (ID: 1VEaM3Ltz5ZS87EbeFsgkDC6aM0s2iivy, 이름: sample2)
파일 압축 해제 중... (파일 이름: sample2)
파일이 'extracted_files/sample2' 디렉토리에 성공적으로 압축 해제되었습니다. (파일 이름: sample2)
파일 다운로드 중... (ID: 1jeib4BD2aG38F0CPi053A1cTMjrgK4Aa, 이름: page1_select_genre.zip)
파일 압축 해제 중... (파일 이름: page1_select_genre.zip)
파일이 'extracted_files/page1_select_genre' 디렉토리에 성공적으로 압축 해제되었습니다. (파일 이름: page1_select_genre.zip)
파일 다운로드 중... (ID: 1K4FwYcpc-UBvkStpaUsW0j4clVAomzc6, 이름: page2_select_setting.zip)
파일 압축 해제 중... (파일 이름: page2_select_setting.zip)
파일이 'extracted_files/page2_select_setting' 디렉토리에 성공적으로 압축 해제되었습니다. (파일 이름: page2_select_setting.zip)
파일 다운로드 중... (ID: 16oiuQogYqvquhRkqc44hQpIbVaXVh1uj, 이름: page3_select_drawing_style.zip)
파일 압축 해제 중... (파일 이름: page3_select_drawing_style.zip)
파일이 'extracted_files/page3_select_drawing_style' 디렉토리

## API 설정

In [ ]:
## API 설정
openai_api_key = os.getenv("OPENAI_API_KEY", "여기에 API 키를 입력하세요.")
client = OpenAI(api_key=openai_api_key)

## 프로그램 실행

In [ ]:
# 1단계: 장르 선택 이미지 경로 설정
gener_imagefile_dict = {
    "동화": "/content/extracted_files/page1_select_genre/fairytale.webp",
    "모험": "/content/extracted_files/page1_select_genre/adventure.webp",
    "판타지": "/content/extracted_files/page1_select_genre/fantasy.webp",
    "미스터리": "/content/extracted_files/page1_select_genre/mystery.webp",
    "SF": "/content/extracted_files/page1_select_genre/sf.webp",
    "슈퍼히어로": "/content/extracted_files/page1_select_genre/superhero.webp",
    "동물 이야기": "/content/extracted_files/page1_select_genre/animal.webp",
    "무서운": "/content/extracted_files/page1_select_genre/scary.webp"
}

# 2단계: 설정 선택 이미지 경로 설정
settings_imagefile_dict = {
    "무작위": "/content/extracted_files/page2_select_setting/Random.webp",
    "성": "/content/extracted_files/page2_select_setting/Castle.webp",
    "학교": "/content/extracted_files/page2_select_setting/School.webp",
    "정글": "/content/extracted_files/page2_select_setting/Jungle.webp",
    "농장": "/content/extracted_files/page2_select_setting/Farm.webp",
    "도시": "/content/extracted_files/page2_select_setting/City.webp",
    "사막": "/content/extracted_files/page2_select_setting/Desert.webp",
    "산악": "/content/extracted_files/page2_select_setting/Mountains.webp",
    "바다": "/content/extracted_files/page2_select_setting/Ocean.webp",
    "숲": "/content/extracted_files/page2_select_setting/Forest.webp",
    "우주": "/content/extracted_files/page2_select_setting/Space.webp",
    "스몰 타운": "/content/extracted_files/page2_select_setting/SmallTown.webp",
    "섬": "/content/extracted_files/page2_select_setting/Island.webp",
    "눈": "/content/extracted_files/page2_select_setting/Snow.webp",
    "언더그라운드": "/content/extracted_files/page2_select_setting/Underground.webp",
    "마을": "/content/extracted_files/page2_select_setting/Village.webp",
    "놀이공원": "/content/extracted_files/page2_select_setting/AmusementPark.webp",
    "동물원": "/content/extracted_files/page2_select_setting/Zoo.webp",
    "미래 도시": "/content/extracted_files/page2_select_setting/FuturisticCity.webp",
    "고대 도시": "/content/extracted_files/page2_select_setting/AncientCity.webp"
}

# 3단계: 그림체 선택 이미지 경로 설정
drawing_style_imagefile_dict = {
    "만화": "/content/extracted_files/page3_select_drawing_style/cartoon.webp",
    "수채화": "/content/extracted_files/page3_select_drawing_style/watercolor.webp",
    "유화": "/content/extracted_files/page3_select_drawing_style/oil_painting.webp",
    "픽셀 아트": "/content/extracted_files/page3_select_drawing_style/pixel_art.webp",
    "연필 스케치": "/content/extracted_files/page3_select_drawing_style/pencil_sketch.webp",
    "애니메이션": "/content/extracted_files/page3_select_drawing_style/animation_style.webp",
    "실사": "/content/extracted_files/page3_select_drawing_style/realistic.webp",
    "웹툰": "/content/extracted_files/page3_select_drawing_style/webtoon.webp"
}


#################### [1단계: 장르 선택 관련 함수] ####################
def select_genre(genre):
    # 1단계에서 사용: 장르 선택
    message = f"당신은 '{genre}' 장르를 선택했습니다. 다음 단계로 진행하세요."
    return genre, message

#################### [2단계: 설정 선택 관련 함수] ####################
def select_setting(setting):
    # 2단계에서 사용: 설정 선택
    message = f"당신은 '{setting}' 설정을 선택했습니다. 다음 단계로 진행하세요."
    return setting, message

#################### [3단계: 그림체 선택 관련 함수] ####################
def select_drawing_style(style):
    # 3단계에서 사용: 그림체 선택
    message = f"당신은 '{style}' 그림체를 선택했습니다. 다음 단계로 진행하세요."
    return style, message

#################### [4단계: 캐릭터 선택 관련 함수] ####################
def add_main_character(name, gender, age, traits):
    # 4단계에서 사용: 주인공 추가
    character_info = {
        "name": name,
        "gender": gender,
        "age": age,
        "traits": traits
    }
    message = f"주인공 추가됨: 이름: {name}, 성별: {gender}, 나이: {age}, 특성: {traits}"
    return character_info, message

def random_character():
    # 4단계에서 사용: 랜덤 캐릭터 생성
    name = "김철수"
    gender = "남성"
    age = "25"
    traits = "용감한, 친절한"
    return name, gender, age, traits

#################### [5단계: 스토리 아이디어 입력 관련 함수] ####################
def submit_story_ideas(story_idea):
    # 5단계에서 사용: 스토리 아이디어 제출
    message = f"스토리 아이디어가 제출되었습니다: {story_idea}"
    return story_idea, message

#################### [6단계: 이야기 길이 선택 관련 함수] ####################
def select_story_length(length):
    # 6단계에서 사용: 이야기 길이 선택
    message = f"선택된 이야기 길이: {length}"
    return length, message

#################### [7단계: 이야기 및 이미지 생성 관련 함수] ####################

def encode_image(image):
    # 7단계에서 사용: 이미지 인코딩
    if image is None:
        raise ValueError("이미지가 없습니다.")
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

def format_character_info(character_info):
    # 7단계에서 사용: 캐릭터 정보 포맷팅
    if character_info:
        return f"이름: {character_info.get('name', '없음')}, 성별: {character_info.get('gender', '없음')}, 나이: {character_info.get('age', '없음')}, 특성: {character_info.get('traits', '없음')}"
    else:
        return "없음"

def generate_image_description(image):
    # 7단계에서 사용: 이미지 설명 생성
    max_retries = 5
    for attempt in range(max_retries):
        try:
            base64_image = encode_image(image)
            image_url = f"data:image/jpeg;base64,{base64_image}"
            description_prompt = """제공된 사진을 아래 요소들을 기반으로 200자 이내로 사실적으로 묘사해
배경, 사진 속 인물(또는 사물)의 위치, 특징적 형태, 표정, 크기, 자세, 주변 배경, 색감, 빛의 방향과 강도, 분위기"""
            start_time = time.time()
            response_text = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": f"{description_prompt}"
                            },
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"{image_url}",
                                },
                            },
                        ],
                    }
                ],
            )
            end_time = time.time()
            duration = end_time - start_time
            print(f"사진 설명 생성 시간: {duration:.2f}초")
            output_text = response_text.choices[0].message.content
            return output_text

        except Exception as e:
            print(f"오류 발생 (시도 {attempt + 1}/{max_retries}): {e}")
            if attempt == max_retries - 1:
                print("최대 재시도 횟수에 도달했습니다. 이미지 설명을 생성하지 못했습니다.")
                return None
            else:
                print("재시도 중입니다...\n")

def generate_text_from_image(image, genre, setting, character_info, story_idea, story_length, chapter_number, image_description, previous_story=None):
    # 7단계에서 사용: 업로드한 이미지 기반으로 이야기 생성

    if image_description is None:
        print("이미지 설명 생성에 실패하여 이야기를 생성할 수 없습니다.")
        return None, None

    base64_image = encode_image(image)
    genre_text = genre if genre else "무작위"
    setting_text = setting if setting else "무작위"
    character_text = format_character_info(character_info)
    story_idea_text = story_idea if story_idea else "없음"
    story_length_text = story_length if story_length else "일반적인 구성의 길이, 약 750-1050 단어"
    previous_story_text = previous_story if previous_story else "없음"

    max_retries = 5
    for attempt in range(max_retries):
        try:
            system_prompt = """
당신은 창의적인 이야기 작가입니다. 주어진 사진과 사진 설명, 지시사항, 주의사항를 바탕으로 흥미롭고 매력적인 이야기를 작성해주세요.
"""
            user_prompt = f"""
[사진 설명]
{image_description}

[지시사항]
- 아래의 정보를 참고하여 이야기를 작성해주세요.
- 업로드된 사진의 세부사항을 이야기 속에 자연스럽게 녹여내어 자세히 묘사해주세요.
- 이야기는 선택한 장르와 설정에 맞는 분위기와 톤을 가져야 합니다.
- 이번에 작성할 내용은 이야기의 다음 장(제 {chapter_number}장)입니다. 각 이미지마다 하나의 장이 생성됩니다.
- 이야기에 반드시 포함되어야 할 요소:
  - 장르: {genre_text}
  - 설정: {setting_text}
  - 주인공 정보: {character_text}
  - 이야기 아이디어: {story_idea_text}
- 이야기의 길이는 {story_length_text}에 맞게 작성해주세요.

[이전 이야기]
{previous_story_text}

[주의사항]
- 이전 이야기가 있다면, 새로운 장이 이전 내용과 자연스럽게 이어지도록 작성해주세요.
- 이전 이야기에서 등장한 캐릭터는 동일한 특성과 배경을 유지하도록 해주세요.
- 각 장의 시작 부분에 "제 {chapter_number}장"을 표기해주세요.
- 상표명이나 특정 기관명은 일반적인 표현으로 변경해주세요.
- 부적절하거나 불쾌감을 줄 수 있는 내용은 피해주세요.
- 문학적인 표현과 풍부한 묘사를 사용하여 독자의 상상력을 자극해주세요.
"""
            print(f"스토리 생성 프롬프트{user_prompt}")
            messages = [
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}",
                            },
                        },
                        {
                            "type": "text",
                            "text": user_prompt
                        },
                    ],
                },
            ]

            start_time = time.time()
            response_text = client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
            )
            end_time = time.time()
            duration = end_time - start_time
            print(f"이야기 생성 시간: {duration:.2f}초")

            output_text = response_text.choices[0].message.content
            return output_text

        except Exception as e:
            print(f"오류 발생 (시도 {attempt + 1}/{max_retries}): {e}")
            if attempt == max_retries - 1:
                print("최대 재시도 횟수에 도달했습니다. 이야기를 생성하지 못했습니다.")
                return None
            else:
                print("재시도 중입니다...\n")

def generate_image_only_from_story(previous_chapters_story, genre, setting, character_info, story_idea, story_length, image_description, drawing_style):
    # 7단계에서 사용: 생성된 이야기에 기반하여 새로운 이미지 생성
    genre_text = genre if genre else "무작위"
    setting_text = setting if setting else "무작위"
    character_text = format_character_info(character_info)
    drawing_style_text = drawing_style if drawing_style else "자유롭게"

    max_retries = 5
    for attempt in range(max_retries):
        try:
            user_prompt = f"""
[사진 설명]
{image_description}
[지시사항]
- 당신은 뛰어난 삽화가이다.
- 위의 **사진 설명**을 기반으로 고품질의 삽화를 만들어
- 그림에는 다음 요소를 고려해야함
  - 장르: {genre_text}
  - 설정: {setting_text}
  - 그림체: {drawing_style_text}
  - 주인공 설정: {character_text}
- 그림의 분위기와 스타일은 반드시 위에 표기된 장르, 설정, 그림체, 주인공설정을 반영해야함
[주의사항]
- 주인공의 특징과 감정을 시각적으로 잘 나타내주세요
- 색상과 조명 등을 활용하여 분위기를 효과적으로 전달해
- 이미지에 텍스트나 글자는 그림에 포함하지 마
"""
            print(f"이미지 생성 프롬프트{user_prompt}")
            start_time = time.time()
            response_generated_image = client.images.generate(
                model="dall-e-3",
                prompt=user_prompt,
                size="1792x1024",
                quality="hd",
                n=1,
            )
            end_time = time.time()
            duration = end_time - start_time
            print(f"이미지 생성 시간: {duration:.2f}초")

            response = requests.get(response_generated_image.data[0].url)
            if response.status_code == 200:
                return Image.open(BytesIO(response.content))
            else:
                print(f"이미지 다운로드 실패. 상태 코드: {response.status_code}")
                return None

        except Exception as e:
            print(f"오류 발생 (시도 {attempt + 1}/{max_retries}): {e}")
            if attempt == max_retries - 1:
                print("최대 재시도 횟수에 도달했습니다. 이미지를 생성하지 못했습니다.")
                return None
            else:
                print("재시도 중입니다...\n")

async def handle_image_upload(image, story_state, image_state, genre_state, setting_state, character_state, story_idea_state, story_length_state, chapters_state, drawing_style_state):

    # 7단계에서 사용: 이미지 업로드 시 이야기+이미지 생성
    if image is None:
        return "이미지를 업로드하세요.", None, story_state, image_state, [], None, None, chapters_state

    chapter_number = len(chapters_state) + 1
    previous_chapter = chapters_state[-1] if chapters_state else None

    start_time = time.time()
    image_description = generate_image_description(image)

    # 두 함수를 동시에 실행 (동기 함수를 to_thread로 비동기화)
    output_text, generated_image = await asyncio.gather(
        asyncio.to_thread(
            generate_text_from_image,
            image,
            genre_state,
            setting_state,
            character_state,
            story_idea_state,
            story_length_state,
            chapter_number,
            image_description,
            previous_chapter
        ),
        asyncio.to_thread(
            generate_image_only_from_story,
            previous_chapter,
            genre_state, setting_state,
            character_state,
            story_idea_state,
            story_length_state,
            image_description,
            drawing_style_state
        )
    )
    end_time = time.time()
    duration = end_time - start_time
    print(f"스토리+이미지 총 생성 시간: {duration:.2f}초")

    if output_text is None or generated_image is None:
        return "이야기 생성에 실패했습니다.", None, story_state, image_state, [], None, None, chapters_state

    image_description_state = image_description
    new_chapters_state = chapters_state + [output_text]
    new_story_state = '\n'.join(new_chapters_state)
    new_previous_story_state = output_text
    new_image_state = image_state + [generated_image] if generated_image else image_state
    gallery_images = [img for img in new_image_state if img is not None]

    return new_story_state.replace('\\n', '\n'), generated_image, new_story_state, new_image_state, gallery_images, image_description_state, new_previous_story_state, new_chapters_state

async def regenerate_story(image_state, story_state, previous_story_state, genre_state, setting_state, character_state, story_idea_state, story_length_state, image_description_state, chapters_state, drawing_style_state):
    # 7단계에서 사용: 스토리 재생성
    if not image_state:
        return "이미지가 없습니다.", None, story_state, image_state, [], image_description_state, previous_story_state, chapters_state

    last_image = image_state[-1]
    chapter_number = len(chapters_state)
    previous_chapter = chapters_state[-2] if len(chapters_state) > 1 else None
    image_description = image_description_state

    # 두 함수 동시 실행
    output_text, generated_image = await asyncio.gather(
        asyncio.to_thread(
            generate_text_from_image,
            last_image,
            genre_state,
            setting_state,
            character_state,
            story_idea_state,
            story_length_state,
            chapter_number,
            image_description,
            previous_story=previous_chapter
        ),
        asyncio.to_thread(
            generate_image_only_from_story,
            previous_chapter,
            genre_state,
            setting_state,
            character_state,
            story_idea_state,
            story_length_state,
            image_description,
            drawing_style_state
        )
    )

    if generated_image is None or output_text is None:
        return "이야기 재생성에 실패했습니다.", None, story_state, image_state, [], image_description_state, previous_story_state, chapters_state

    new_chapters_state = chapters_state[:-1] + [output_text]
    new_story_state = '\n'.join(new_chapters_state)
    new_image_description_state = image_description
    new_image_state = image_state.copy()
    if generated_image:
        new_image_state[-1] = generated_image

    gallery_images = [img for img in new_image_state if img is not None]

    return new_story_state.replace('\\n', '\n'), generated_image, new_story_state, new_image_state, gallery_images, new_image_description_state, output_text, new_chapters_state

def regenerate_image(story_state, previous_story_state, genre_state, setting_state, character_state, story_idea_state, story_length_state, image_state, image_description_state, drawing_style_state):
    # 7단계에서 사용: 이미지 재생성
    generated_image = generate_image_only_from_story(previous_story_state, genre_state, setting_state, character_state, story_idea_state, story_length_state, image_description_state, drawing_style_state)

    new_image_state = image_state.copy()
    if generated_image is not None:
        new_image_state[-1] = generated_image

    gallery_images = [img for img in new_image_state if img is not None]

    return generated_image, new_image_state, gallery_images

#################### [최종: 내보내기 및 공유 관련 함수] ####################

def export_to_pdf(story_text, image_list):
    # 완성된 이야기 PDF 내보내기
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_font('NotoSans', '', 'NotoSansCJKkr-Regular.otf', uni=True)
    pdf.set_font('NotoSans', size=12)

    story_paragraphs = story_text.split('\n')
    for idx in range(max(len(image_list), len(story_paragraphs))):
        pdf.add_page()
        if idx < len(image_list) and image_list[idx]:
            img_path = f"temp_image_{idx}.png"
            image_list[idx].save(img_path)
            pdf.image(img_path, x=10, y=10, w=pdf.w - 20)
            os.remove(img_path)
            pdf.ln(85)
        if idx < len(story_paragraphs):
            pdf.multi_cell(0, 10, story_paragraphs[idx], align='L')

    pdf_output = io.BytesIO()
    pdf.output(pdf_output)
    pdf_output.seek(0)
    return pdf_output

def export_to_word(story_text, image_list):
    # 완성된 이야기 Word 내보내기
    doc = docx.Document()
    story_paragraphs = story_text.split('\n')

    for idx in range(max(len(image_list), len(story_paragraphs))):
        if idx < len(image_list) and image_list[idx]:
            img_path = f"temp_image_{idx}.png"
            image_list[idx].save(img_path)
            doc.add_picture(img_path, width=docx.shared.Inches(6))
            os.remove(img_path)
        if idx < len(story_paragraphs):
            doc.add_paragraph(story_paragraphs[idx])

    word_output = io.BytesIO()
    doc.save(word_output)
    word_output.seek(0)
    return word_output

def share_on_kakaotalk(story_text):
    # 완성된 이야기 카카오톡 공유
    share_text = f"나만의 이야기:\n{story_text}"
    kakao_link = f"https://sharer.kakao.com/talk/friends/picker/link?link_ver=4&text={requests.utils.quote(share_text)}"
    webbrowser.open(kakao_link)

def share_on_twitter(story_text):
    # 완성된 이야기 트위터 공유
    share_text = f"나만의 이야기:\n{story_text}"
    twitter_link = f"https://twitter.com/intent/tweet?text={requests.utils.quote(share_text)}"
    webbrowser.open(twitter_link)

#################### UI 및 실행 부분 ####################
with gr.Blocks(css=".gradio-container footer {display: none !important;}") as demo:

    # 상태 변수들
    genre_state = gr.State()
    setting_state = gr.State()
    character_state = gr.State()
    story_idea_state = gr.State()
    story_length_state = gr.State()
    previous_story_state = gr.State("")
    image_description_state = gr.State()
    image_state = gr.State([])  # 이미지 리스트로 초기화
    story_state = gr.State("")  # 이야기 문자열로 초기화
    chapters_state = gr.State([])  # 각 장의 내용을 저장할 리스트
    drawing_style_state = gr.State()  # 그림체 상태 변수 추가

    #################### 탭 구성 시작 ####################

    with gr.Tabs(elem_id="tabs") as tabs:
        # 프로젝트 소개 탭
        with gr.TabItem("프로젝트 소개"):

            # 시작하기 버튼 (상단)
            start_button_top = gr.Button("시작하기", variant="primary")
            start_button_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 1) {
                            buttons[1].click();
                        }
                    }
                }
                """
            )

            # 프로젝트 소개 섹션
            gr.Image(value="https://drive.google.com/uc?id=10WVpFeZ3_8KxfBWg5OZG-yKUQY0z9Au2")
            gr.Markdown("# 프로젝트 소개")
            gr.Markdown("""
            [프로젝트 레포트 보기](https://drive.google.com/uc?id=1MN97nvxdc4gzdN486hdPiO25bEw07cPj)
            [샘플 파일 다운로드](https://drive.google.com/uc?id=1VEaM3Ltz5ZS87EbeFsgkDC6aM0s2iivy)
            """)
            gr.Markdown("""
            ## 나의 사진으로 나만의 이야기 만들기
            ### AI 기반 인터랙티브 스토리 창작 플랫폼
                본 프로젝트는 사용자의 사진과 상상력을 AI로 결합하여
                개인화된 스토리를 창작하고, 이를 시각적 이미지로 재현하는
                새로운 디지털 콘텐츠 경험을 제공하는 플랫폼입니다.
            ---
            """)

            gr.Image(value="https://drive.google.com/uc?id=1ufnovxeKPbNoulGPDIgmjrsWtIRc_YCq")
            gr.Markdown("""
            ## 프로젝트 개요
            ### 프로젝트 배경
            디지털 시대, 사람들은 사진·영상·텍스트 등 다양한 미디어로 순간을 기록하고 공유합니다.
            그러나 단순 감상을 넘어, 사진 속 기억과 감정을 확장하고자 하는 욕구가 커지고 있습니다.
            본 프로젝트는 이러한 니즈를 반영하여 **ChatGPT** 등의 대규모 언어 모델(LLM)과
            이미지 생성 AI를 활용, 사용자가 **자신의 사진 + 상상력**으로 독창적 스토리를 만들어낼 수 있는
            인터랙티브 플랫폼을 제안합니다.
            """)
            gr.Markdown("""
            ### 목표 및 기대 효과
            - **개인화된 스토리텔링 경험**: 사진·장르·배경·그림체·캐릭터·길이 등 사용자 주도형 설정
            - **창의성 촉진**: 사진 기반 스토리 재해석으로 창의적 사고와 몰입도 상승
            - **기술 접근성 향상**: AI 활용을 직관적 UI/UX로 구현, 누구나 쉽게 사용 가능
            - **새로운 디지털 콘텐츠 생태계**: 교육, 문화예술, 마케팅 등 다양한 분야로 확장 가능
            ---
            """)

            gr.Image(value="https://drive.google.com/uc?id=1m0FzoDf5lpKhjQI_0YA0hdoau-IZYnTZ")
            gr.Image(value="https://drive.google.com/uc?id=1EoiTjcU_4xTnckslokMv_qUxwDtJMe17")
            gr.Markdown("""
            ## AI 기반 스토리 및 이미지 생성 프로세스
            위 그림은 프로젝트 내 AI 활용 흐름을 단계별로 보여줍니다:
                이미지 업로드 → 이미지 설명 생성 → 스토리 프롬프트 구성 → 스토리 생성 → 일러스트 생성
            """)
            gr.Markdown("""
            ## 주요 기능
            1. **사진 업로드 및 분석**
            - 사용자 사진을 AI 모델로 분석하여 텍스트 묘사 생성
            - 스토리 생성 프롬프트에 활용

            2. **장르·배경·그림체·캐릭터 설정**
            - 장르: 동화, 판타지, SF, 미스터리 등 다양
            - 배경: 성, 숲, 도시, 우주 등 원하는 환경 지정
            - 그림체: 만화풍, 수채화, 픽셀아트 등 스타일 선택
            - 캐릭터: 이름, 특징, 나이, 성별 등 자유롭게 설정

            3. **스토리 아이디어 입력 및 길이 조정**
            - 사용자 입력 아이디어 또는 특정 설정 전달
            - 짧은 단편부터 긴 서사까지 분량 선택

            4. **AI 기반 스토리·이미지 생성**
            - ChatGPT(LLM) 기반 맞춤형 스토리 자동 생성
            - Stable Diffusion, DALL·E 등으로 스토리 삽화 자동 생성
            - 재생성 기능으로 원하는 결과 도달 가능

            5. **결과물 공유 및 내보내기**
            - 완성된 스토리+이미지를 PDF, Word 형태로 다운로드
            - 카카오톡, 트위터 등 SNS 공유 지원

            """)


            gr.Image(value="https://drive.google.com/uc?id=1RO47hCP2SybG-BHPOTxabcd9_gSyzXq4")
            gr.Markdown("""
            ## 인터페이스 및 사용자 흐름
                사용자는 장르 선택 → 설정 선택 → 그림체 선택 → 이야기 아이디어 입력 → 이야기 길이 선택 → 이미지 업로드 → 이야기 및 이미지 생성순으로 진행합니다.
            ---
            """)

            gr.Markdown("""
            ## 구현 방안 및 기술 스택
            - **프론트엔드/UI**: Gradio 기반 웹 인터페이스
            직관적이며 단계별 의사결정 흐름 지원
            - **백엔드 로직**: Python 기반 서버
            이미지 처리, API 연동, 상태 관리 수행
            - **AI 모델 연계**: OpenAI API(ChatGPT), Stable Diffusion, CLIP/BLIP 등
            Vision-Language 모델 활용
            - **확장성**: 모듈화된 API 구조로 다양한 모델 및 서비스 연계 가능
            ---
            """)

            gr.Image(value="https://drive.google.com/uc?id=12N9g_X_qP4fVywvLVNKdTkxfWhIx-K7O")
            gr.Markdown("""
            ## 초기 전략 및 성장 방안
            - **마이크로 마켓 전략**: 교육용 맞춤형 동화, 특정 장르 마니아층 등 틈새시장 공략
            - **단면 우선 전략**: 핵심 기능(사진 기반 스토리) 중심 MVP 출시 후 점진적 확장
            - **생산자 주도 전파 전략**: 사용자 창작물 SNS 공유 유도, 자발적 입소문 형성
            ---
            """)

            gr.Image(value="https://drive.google.com/uc?id=1QRT2rbR6fPHvwyzXyNkKfrtv0r_MBuzt")
            gr.Markdown("""
            ## 기대 효과
            - **몰입형 디지털 스토리텔링 경험**: 사진과 AI가 결합된 새로운 콘텐츠 제공
            - **사용자 창의성 극대화**: 즉각적 피드백·재생성을 통한 풍부한 창작 과정
            - **지속 가능한 생태계 조성**: 교육, 마케팅, 예술 등 다방면 활용·확장 가능
            ---
            """)

            gr.Markdown("""
            ## 결론 및 향후 과제

            본 프로젝트는 개인화된 인터랙티브 스토리텔링의 새로운 패러다임을 제안합니다.
            향후 모델 고도화, UI/UX 개선, 교육·문화 예술 분야 협업, 수익화 모델 검토 등을 통해
            한 단계 더 발전시켜나갈 예정입니다.
            """)

            # 하단 시작하기 버튼
            start_button_bottom = gr.Button("시작하기", variant="primary")
            start_button_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 1) {
                            buttons[1].click();
                        }
                    }
                }
                """
            )

        # 1단계: 장르 선택
        with gr.TabItem("장르 선택"):
            gr.Markdown("# 새로운 이야기를 만들어보세요")
            gr.Markdown("## 당신의 이야기를 시작하세요: 장르를 선택하세요")
            gr.Markdown("모든 훌륭한 이야기는 장르에서 시작됩니다. 당신의 모험에 맞는 분위기와 스타일을 설정할 장르를 선택하세요.")

            # 상단 뒤로가기, 다음으로 가기 버튼 (1단계는 뒤로가기 비활성화)
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn1_top = gr.Button("뒤로가기", variant="secondary", interactive=False)
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn1_top = gr.Button("다음으로 가기", variant="primary")

            next_btn1_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 1) {
                            buttons[2].click();
                        }
                    }
                }
                """
            )

            # 선택된 장르를 표시할 텍스트박스
            selected_genre_text = gr.Textbox(label="선택된 장르", interactive=False)

            # 장르 목록과 설명
            genres = ["동화", "모험", "판타지", "미스터리", "SF", "슈퍼히어로", "동물 이야기", "무서운"]
            genre_descriptions = [
                "마법, 왕자와 공주, 도덕적 교훈이 자주 등장하는 전통적인 이야기입니다.",
                "이국적이거나 도전적인 환경에서의 흥미진진한 여행, 퀘스트 또는 탐험이 포함된 이야기입니다.",
                "마법 요소, 신화적인 생물, 환상적인 세계를 포함한 이야기입니다.",
                "캐릭터가 퍼즐을 풀거나, 미스터리를 해결하거나, 비밀을 밝히는 이야기입니다.",
                "미래 기술, 우주 탐사, 시간 여행 및 기타 과학적 개념에 중점을 둡니다.",
                "비범한 능력이나 힘을 가진 캐릭터가 악과 싸우는 이야기를 다룹니다.",
                "주요 캐릭터가 인간적인 특성을 지닌 동물이며, 그들의 모험이나 상호 작용을 탐구합니다.",
                "유령, 몬스터, 좀비가 등장할 수 있으며 약간의 긴장감을 주는 무서운 이야기입니다."
            ]

            with gr.Row():
                for i, genre in enumerate(genres):
                    with gr.Column():
                        image_path = gener_imagefile_dict.get(genre)
                        if image_path and os.path.exists(image_path):
                            img = Image.open(image_path)
                            gr.Image(img, label=genre, elem_id=f"{genre.lower().replace(' ', '_')}_image")
                        else:
                            gr.Markdown(f"**이미지를 사용할 수 없습니다**")

                        gr.Markdown(f"**{genre}**\n\n{genre_descriptions[i]}")
                        btn = gr.Button("선택")
                        btn.click(fn=lambda g=genre: select_genre(g), inputs=None, outputs=[genre_state, selected_genre_text], queue=False)

            # 하단 뒤로가기, 다음으로 가기 버튼 (1단계는 뒤로가기 비활성화)
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn1_bottom = gr.Button("뒤로가기", variant="secondary", interactive=False)
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn1_bottom = gr.Button("다음으로 가기", variant="primary")

            next_btn1_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 1) {
                            buttons[2].click();
                        }
                    }
                }
                """
            )


        # 2단계: 설정 선택
        with gr.TabItem("설정 선택"):
            gr.Markdown("## 장면 설정: 설정을 선택하세요")
            gr.Markdown("이야기가 어디에서 펼쳐지나요? 도시의 활기찬 거리부터 먼 행성의 고요한 풍경까지, 당신의 이야기에 생명을 불어넣는 설정을 선택하세요.")

            selected_setting_text = gr.Textbox(label="선택된 설정", interactive=False)

            # 상단 뒤로가기, 다음으로 가기 버튼
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn2_top = gr.Button("뒤로가기", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn2_top = gr.Button("다음으로 가기", variant="primary")

            back_btn2_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 0) {
                            buttons[1].click();
                        }
                    }
                }
                """
            )
            next_btn2_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 2) {
                            buttons[3].click();
                        }
                    }
                }
                """
            )

            settings = [
                "무작위", "성", "학교", "정글", "농장", "도시", "사막", "산악",
                "바다", "숲", "우주", "스몰 타운", "섬", "눈",
                "언더그라운드", "마을", "놀이공원", "동물원", "미래 도시", "고대 도시"
            ]
            setting_descriptions = [
                "미리 정의된 이야기 설정 중 하나를 무작위로 선택합니다 (성, 섬, 놀이공원 등).",
                "왕국 또는 중세 테마의 역사적 또는 환상적인 설정으로, 동화나 판타지 이야기에 자주 사용됩니다.",
                "우정, 학습, 개인 성장에 관한 이야기에 적합한 학습 공간입니다.",
                "이국적인 동식물이 가득한 밀집된 열대 환경으로, 모험과 탐험 이야기에 적합합니다.",
                "자연과 농업의 삶을 강조하는 들판, 헛간, 동물이 있는 시골 환경입니다.",
                "고층 빌딩과 다양한 커뮤니티가 있는 도시 환경으로, 빠르게 진행되는 현대적 라이프스타일을 반영합니다.",
                "극한의 온도를 가진 건조하고 모래로 가득한 환경으로, 생존 및 모험 이야기에 배경을 제공합니다.",
                "험준한 지형을 가진 고산 설정으로, 모험과 탐험 테마에 자주 사용됩니다.",
                "해변, 외딴 또는 수중 환경을 포함한 해양 환경으로 탐험과 모험에 이상적입니다.",
                "동화나 모험 이야기에 완벽한 매혹적이거나 신비로운 숲의 울창한 지역입니다.",
                "우주선이나 다른 행성에 있는 외계 환경으로, 공상과학이나 미래 이야기에 적합합니다.",
                "친밀한 커뮤니티 환경으로, 고풍스러운 분위기도 풍기며 커뮤니티와 관계에 대한 이야기에 이상적입니다.",
                "외딴 고립된 환경으로 모험, 생존 또는 미스터리 스토리에 완벽합니다.",
                "극지방이나 눈 덮인 산맥 같이 춥고 많은 환경으로 생존과 모험 이야기 배경에 적합합니다.",
                "동굴이나 터널과 같은 지하 환경은 신비롭고 모험적인 분위기를 줍니다.",
                "지역 생물, 커뮤니티, 단순한 이야기가 배경이 되는 작고 조용히 사는 커뮤니티입니다.",
                "놀이기구와 볼거리가 있는 재미있고 흥미로운 환경으로 모험과 유머러스한 이야기에 이상적입니다.",
                "교육용, 동물 기반 또는 모험 이야기에 적합한 다양한 동물이 있는 환경입니다.",
                "첨단 기술 기능을 갖춘 고급 도시 환경으로, 공상 과학이나 디스토피아적 이야기에 적합합니다.",
                "과거로 돌아가 고대 도시의 신비와 경이로움을 탐험하세요."
            ]

            with gr.Row():
                for i, setting in enumerate(settings):
                    with gr.Column():
                        image_path = settings_imagefile_dict.get(setting)
                        if image_path and os.path.exists(image_path):
                            img = Image.open(image_path)
                            gr.Image(img, label=setting)
                        else:
                            gr.Markdown("**이미지를 사용할 수 없습니다**")

                        gr.Markdown(f"**{setting}**\n\n{setting_descriptions[i]}")
                        btn = gr.Button("선택")
                        btn.click(fn=lambda s=setting: select_setting(s), inputs=None, outputs=[setting_state, selected_setting_text], queue=False)

            # 하단 뒤로가기, 다음으로 가기 버튼
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn2_bottom = gr.Button("뒤로가기", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn2_bottom = gr.Button("다음으로 가기", variant="primary")

            back_btn2_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 0) {
                            buttons[1].click();
                        }
                    }
                }
                """
            )

            next_btn2_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 2) {
                            buttons[3].click();
                        }
                    }
                }
                """
            )


        # 3단계: 그림체 선택
        with gr.TabItem("그림체 선택"):
            gr.Markdown("## 그림체 선택")
            gr.Markdown("이야기에 사용될 그림체를 선택하세요.")
            selected_style_text = gr.Textbox(label="선택된 그림체", interactive=False)

            # 상단 뒤로가기, 다음으로 가기 버튼
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn_style_top = gr.Button("뒤로가기", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn_style_top = gr.Button("다음으로 가기", variant="primary")

            back_btn_style_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 1) {
                            buttons[2].click();
                        }
                    }
                }
                """
            )
            next_btn_style_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 3) {
                            buttons[4].click();
                        }
                    }
                }
                """
            )

            drawing_styles = ["만화", "수채화", "유화", "픽셀 아트", "연필 스케치", "애니메이션", "실사", "웹툰"]
            style_descriptions = [
                "밝고 생생한 색상과 단순화된 인물 묘사.",
                "부드럽고 흐릿한 가장자리의 수채화.",
                "두꺼운 질감과 풍부한 색감의 유화.",
                "고전 게임 스타일의 픽셀 아트.",
                "세밀한 연필 선 스케치.",
                "생동감 있고 표현력 풍부한 애니메이션 스타일.",
                "실제와 같은 질감의 실사.",
                "현대적이고 디지털한 웹툰 스타일."
            ]

            with gr.Row():
                for i, style in enumerate(drawing_styles):
                    with gr.Column():
                        image_path = drawing_style_imagefile_dict.get(style)
                        if image_path and os.path.exists(image_path):
                            img = Image.open(image_path)
                            gr.Image(img, label=style)
                        else:
                            gr.Markdown("**이미지를 사용할 수 없습니다**")

                        gr.Markdown(f"**{style}**\n\n{style_descriptions[i]}")
                        btn = gr.Button("선택")
                        btn.click(fn=lambda s=style: select_drawing_style(s), inputs=None, outputs=[drawing_style_state, selected_style_text], queue=False)

            # 하단 뒤로가기, 다음으로 가기 버튼
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn_style_bottom = gr.Button("뒤로가기", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn_style_bottom = gr.Button("다음으로 가기", variant="primary")

            back_btn_style_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 1) {
                            buttons[2].click();
                        }
                    }
                }
                """
            )
            next_btn_style_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 3) {
                            buttons[4].click();
                        }
                    }
                }
                """
            )

        # 4단계: 캐릭터 선택
        with gr.TabItem("캐릭터 선택"):
            gr.Markdown("## 캐릭터 추가: 주인공을 만들어보세요")
            gr.Markdown("이야기 속 인물들을 소개하세요. 주인공과 주변 인물들을 추가하고 이들의 특성을 정의해 풍부하고 역동적인 상호작용을 만들어 보세요.")

            # 상단 뒤로가기, 다음으로 가기 버튼
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn3_top = gr.Button("뒤로가기", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn3_top = gr.Button("다음으로 가기", variant="primary")

            back_btn3_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 2) {
                            buttons[3].click();
                        }
                    }
                }
                """
            )
            next_btn3_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 4) {
                            buttons[5].click();
                        }
                    }
                }
                """
            )

            name_input = gr.Textbox(label="이름", placeholder="이름을 입력하세요")
            gender_input = gr.Dropdown(["남성", "여성", "기타"], label="성별")
            age_input = gr.Textbox(label="나이", placeholder="나이를 입력하세요")
            traits_input = gr.Textbox(label="특성", placeholder="특성을 입력하세요 (예: 용감한, 친절한)")

            random_button = gr.Button("랜덤 생성")
            random_button.click(
                fn=random_character,
                inputs=None,
                outputs=[name_input, gender_input, age_input, traits_input]
            )

            character_info_display = gr.Textbox(label="주인공 정보", interactive=False)
            add_character_button = gr.Button("주인공 추가")
            add_character_button.click(
                fn=add_main_character,
                inputs=[name_input, gender_input, age_input, traits_input],
                outputs=[character_state, character_info_display]
            )

            # 하단 뒤로가기, 다음으로 가기 버튼
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn3_bottom = gr.Button("뒤로가기", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn3_bottom = gr.Button("다음으로 가기", variant="primary")

            back_btn3_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 2) {
                            buttons[3].click();
                        }
                    }
                }
                """
            )
            next_btn3_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 4) {
                            buttons[5].click();
                        }
                    }
                }
                """
            )

        # 5단계: 세부 사항 추가
        with gr.TabItem("세부 사항 추가"):
            gr.Markdown("## 당신만의 이야기 만들기: 스토리 아이디어")
            gr.Markdown("### 스토리 아이디어 (선택 사항)")
            gr.Markdown("이야기에서 어떤 특별한 일이 일어나길 원하나요? 재미있는 순간이나 놀라운 반전을 추가할 수 있습니다. 이곳에서 스토리를 여러분의 방식대로 만들어 보세요!")

            # 상단 뒤로가기, 다음으로 가기 버튼
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn4_top = gr.Button("뒤로가기", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn4_top = gr.Button("다음으로 가기", variant="primary")

            back_btn4_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 3) {
                            buttons[4].click();
                        }
                    }
                }
                """
            )
            next_btn4_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 5) {
                            buttons[6].click();
                        }
                    }
                }
                """
            )

            story_idea_input = gr.Textbox(
                placeholder="이야기에서 재미있거나 흥미로운 내용을 입력하세요.",
                lines=4,
                show_label=False
            )
            story_output = gr.Textbox(label="제출된 스토리 아이디어", interactive=False)
            continue_button = gr.Button("계속하기")
            continue_button.click(
                fn=submit_story_ideas,
                inputs=story_idea_input,
                outputs=[story_idea_state, story_output]
            )

            # 하단 뒤로가기, 다음으로 가기 버튼
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn4_bottom = gr.Button("뒤로가기", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn4_bottom = gr.Button("다음으로 가기", variant="primary")

            back_btn4_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 3) {
                            buttons[4].click();
                        }
                    }
                }
                """
            )
            next_btn4_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 5) {
                            buttons[6].click();
                        }
                    }
                }
                """
            )

        # 6단계: 이야기 길이 선택
        with gr.TabItem("이야기 길이 선택"):
            gr.Markdown("## 이야기 길이 선택")
            gr.Markdown("원하는 이야기 길이를 선택하세요.")
            selected_length_text = gr.Textbox(label="선택된 이야기 길이", interactive=False)

            # 상단 뒤로가기, 다음으로 가기 버튼
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn5_top = gr.Button("뒤로가기", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn5_top = gr.Button("다음으로 가기", variant="primary")

            back_btn5_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 4) {
                            buttons[5].click();
                        }
                    }
                }
                """
            )
            next_btn5_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 6) {
                            buttons[7].click();
                        }
                    }
                }
                """
            )

            with gr.Row():
                with gr.Column():
                    gr.Markdown("### 짧은 이야기")
                    gr.Markdown("약 300-450 단어")
                    short_button = gr.Button("선택")
                    short_button.click(
                        fn=lambda: select_story_length("짧은 구성의 길이, 약 300-450 단어"),
                        inputs=[],
                        outputs=[story_length_state, selected_length_text]
                    )

                with gr.Column():
                    gr.Markdown("### 보통 이야기")
                    gr.Markdown("약 750-1050 단어")
                    normal_button = gr.Button("선택")
                    normal_button.click(
                        fn=lambda: select_story_length("일반적인 구성의 길이, 약 750-1050 단어"),
                        inputs=[],
                        outputs=[story_length_state, selected_length_text]
                    )

                with gr.Column():
                    gr.Markdown("### 긴 이야기 🔒")
                    gr.Markdown("약 1500-2250 단어 \n로그인 사용자에게만 제공됩니다.")
                    long_button = gr.Button("선택", interactive=False)

            # 하단 뒤로가기, 다음으로 가기 버튼
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn5_bottom = gr.Button("뒤로가기", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn5_bottom = gr.Button("다음으로 가기", variant="primary")

            back_btn5_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 4) {
                            buttons[5].click();
                        }
                    }
                }
                """
            )
            next_btn5_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 6) {
                            buttons[7].click();
                        }
                    }
                }
                """
            )

        # 7단계: 그림 및 이야기 생성
        with gr.TabItem("그림 및 이야기 생성"):
            gr.Markdown("## 그림 및 이야기를 생성합니다")
            gr.Markdown("이미지를 업로드하면 해당 이미지에 기반한 이야기가 생성되고, 그 이야기로부터 새로운 이미지가 생성됩니다.")

            # 상단 뒤로가기, 다음으로 가기 버튼
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn6_top = gr.Button("뒤로가기", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn6_top = gr.Button("다음으로 가기", variant="primary")

            back_btn6_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 5) {
                            buttons[6].click();
                        }
                    }
                }
                """
            )
            next_btn6_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 7) {
                            buttons[8].click();
                        }
                    }
                }
                """
            )

            image_input = gr.Image(type="pil", label="이미지 업로드")
            image_output = gr.Image(type="pil", label="생성된 이미지")
            text_output = gr.Textbox(label="생성된 이야기", lines=10, interactive=False)
            gallery_output = gr.Gallery(label="이전 생성된 이미지들", columns=4, height='auto')

            image_input.change(
                fn=handle_image_upload,
                inputs=[image_input, story_state, image_state, genre_state, setting_state, character_state, story_idea_state, story_length_state, chapters_state, drawing_style_state],
                outputs=[text_output, image_output, story_state, image_state, gallery_output, image_description_state, previous_story_state, chapters_state],
                show_progress=True
            )

            regenerate_story_btn = gr.Button("스토리 재생성")
            regenerate_image_btn = gr.Button("이미지 재생성")

            regenerate_story_btn.click(
                fn=regenerate_story,
                inputs=[image_state, story_state, previous_story_state, genre_state, setting_state, character_state, story_idea_state, story_length_state, image_description_state, chapters_state, drawing_style_state],
                outputs=[text_output, image_output, story_state, image_state, gallery_output, image_description_state, previous_story_state, chapters_state],
            )

            regenerate_image_btn.click(
                fn=regenerate_image,
                inputs=[story_state, previous_story_state, genre_state, setting_state, character_state, story_idea_state, story_length_state, image_state, image_description_state, drawing_style_state],
                outputs=[image_output, image_state, gallery_output],
            )

            # 하단 뒤로가기, 다음으로 가기 버튼
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn6_bottom = gr.Button("뒤로가기", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn6_bottom = gr.Button("다음으로 가기", variant="primary")

            back_btn6_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 5) {
                            buttons[6].click();
                        }
                    }
                }
                """
            )
            next_btn6_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 7) {
                            buttons[8].click();
                        }
                    }
                }
                """
            )

        # 8단계: 완성된 이야기 확인 및 내보내기
        with gr.TabItem("완성된 이야기 확인 및 내보내기"):
            gr.Markdown("## 완성된 이야기")
            gr.Markdown("생성된 그림과 이야기를 확인하고 저장하거나 공유하세요.")

            # 상단 뒤로가기 (마지막 단계이므로 다음 버튼 없음)
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn7_top = gr.Button("뒤로가기", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")

            back_btn7_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 6) {
                            buttons[7].click();
                        }
                    }
                }
                """
            )

            final_story_output = gr.Textbox(label="완성된 이야기", lines=10, interactive=False)
            final_gallery_output = gr.Gallery(label="생성된 이미지들", columns=4, height='auto')

            def display_final_story(story_state, image_state):
                return story_state, image_state

            display_button = gr.Button("완성된 이야기 보기")
            display_button.click(
                fn=display_final_story,
                inputs=[story_state, image_state],
                outputs=[final_story_output, final_gallery_output]
            )

            pdf_export_button = gr.Button("PDF로 내보내기")
            pdf_file = gr.File(label="다운로드한 PDF 파일")
            pdf_export_button.click(
                fn=export_to_pdf,
                inputs=[story_state, image_state],
                outputs=pdf_file
            )

            word_export_button = gr.Button("Word로 내보내기")
            word_file = gr.File(label="다운로드한 Word 파일")
            word_export_button.click(
                fn=export_to_word,
                inputs=[story_state, image_state],
                outputs=word_file
            )

            kakaotalk_share_button = gr.Button("카카오톡으로 공유하기")
            kakaotalk_share_button.click(
                fn=share_on_kakaotalk,
                inputs=[story_state],
                outputs=None
            )

            twitter_share_button = gr.Button("트위터로 공유하기")
            twitter_share_button.click(
                fn=share_on_twitter,
                inputs=[story_state],
                outputs=None
            )

            # 하단 뒤로가기 (마지막 단계이므로 다음 버튼 없음)
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn7_bottom = gr.Button("뒤로가기", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")

            back_btn7_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 6) {
                            buttons[7].click();
                        }
                    }
                }
                """
            )

    #################### 탭 구성 종료 ####################
    demo.launch(debug=True, show_api=False)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://fb67c2d95e22ee99a6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# 프로젝트 소개 섹션

![프로젝트 소개](https://drive.google.com/uc?id=10WVpFeZ3_8KxfBWg5OZG-yKUQY0z9Au2)

# 프로젝트 소개

## 나의 사진으로 나만의 이야기 만들기
### AI 기반 인터랙티브 스토리 창작 플랫폼

본 프로젝트는 사용자의 사진과 상상력을 AI로 결합하여  
개인화된 스토리를 창작하고, 이를 시각적 이미지로 재현하는  
새로운 디지털 콘텐츠 경험을 제공하는 플랫폼입니다.

---

![프로젝트 개요](https://drive.google.com/uc?id=1ufnovxeKPbNoulGPDIgmjrsWtIRc_YCq)

## 프로젝트 개요
### 프로젝트 배경

디지털 시대, 사람들은 사진·영상·텍스트 등 다양한 미디어로 순간을 기록하고 공유합니다.  
그러나 단순 감상을 넘어, 사진 속 기억과 감정을 확장하고자 하는 욕구가 커지고 있습니다.  
본 프로젝트는 이러한 니즈를 반영하여 **ChatGPT** 등의 대규모 언어 모델(LLM)과  
이미지 생성 AI를 활용, 사용자가 **자신의 사진 + 상상력**으로 독창적 스토리를 만들어낼 수 있는  
인터랙티브 플랫폼을 제안합니다.

### 목표 및 기대 효과
- **개인화된 스토리텔링 경험**: 사진·장르·배경·그림체·캐릭터·길이 등 사용자 주도형 설정
- **창의성 촉진**: 사진 기반 스토리 재해석으로 창의적 사고와 몰입도 상승
- **기술 접근성 향상**: AI 활용을 직관적 UI/UX로 구현, 누구나 쉽게 사용 가능
- **새로운 디지털 콘텐츠 생태계**: 교육, 문화예술, 마케팅 등 다양한 분야로 확장 가능

---

![AI 활용 흐름도 1](https://drive.google.com/uc?id=1m0FzoDf5lpKhjQI_0YA0hdoau-IZYnTZ)
![AI 활용 흐름도 2](https://drive.google.com/uc?id=1EoiTjcU_4xTnckslokMv_qUxwDtJMe17)

## AI 기반 스토리 및 이미지 생성 프로세스

위 그림은 프로젝트 내 AI 활용 흐름을 단계별로 보여줍니다:  
이미지 업로드 → 이미지 설명 생성 → 스토리 프롬프트 구성 → 스토리 생성 → 일러스트 생성

## 주요 기능
1. **사진 업로드 및 분석**  
   - 사용자 사진을 AI 모델로 분석하여 텍스트 묘사 생성  
   - 스토리 생성 프롬프트에 활용

2. **장르·배경·그림체·캐릭터 설정**  
   - 장르: 동화, 판타지, SF, 미스터리 등 다양  
   - 배경: 성, 숲, 도시, 우주 등 원하는 환경 지정  
   - 그림체: 만화풍, 수채화, 픽셀아트 등 스타일 선택  
   - 캐릭터: 이름, 특징, 나이, 성별 등 자유롭게 설정

3. **스토리 아이디어 입력 및 길이 조정**  
   - 사용자 입력 아이디어 또는 특정 설정 전달  
   - 짧은 단편부터 긴 서사까지 분량 선택

4. **AI 기반 스토리·이미지 생성**  
   - ChatGPT(LLM) 기반 맞춤형 스토리 자동 생성  
   - Stable Diffusion, DALL·E 등으로 스토리 삽화 자동 생성  
   - 재생성 기능으로 원하는 결과 도달 가능

5. **결과물 공유 및 내보내기**  
   - 완성된 스토리+이미지를 PDF, Word 형태로 다운로드  
   - 카카오톡, 트위터 등 SNS 공유 지원  

---

![인터페이스 흐름도](https://drive.google.com/uc?id=1RO47hCP2SybG-BHPOTxabcd9_gSyzXq4)

## 인터페이스 및 사용자 흐름

사용자는  
장르 선택 → 설정 선택 → 그림체 선택 → 이야기 아이디어 입력 → 이야기 길이 선택 → 이미지 업로드 → 이야기 및 이미지 생성  
순으로 진행합니다.

---

## 구현 방안 및 기술 스택
- **프론트엔드/UI**: Gradio 기반 웹 인터페이스  
  직관적이며 단계별 의사결정 흐름 지원  
- **백엔드 로직**: Python 기반 서버  
  이미지 처리, API 연동, 상태 관리 수행  
- **AI 모델 연계**: OpenAI API(ChatGPT), Stable Diffusion, CLIP/BLIP 등  
  Vision-Language 모델 활용  
- **확장성**: 모듈화된 API 구조로 다양한 모델 및 서비스 연계 가능  

---

![초기 전략 및 성장 방안](https://drive.google.com/uc?id=12N9g_X_qP4fVywvLVNKdTkxfWhIx-K7O)

## 초기 전략 및 성장 방안
- **마이크로 마켓 전략**: 교육용 맞춤형 동화, 특정 장르 마니아층 등 틈새시장 공략
- **단면 우선 전략**: 핵심 기능(사진 기반 스토리) 중심 MVP 출시 후 점진적 확장
- **생산자 주도 전파 전략**: 사용자 창작물 SNS 공유 유도, 자발적 입소문 형성

---

![기대 효과](https://drive.google.com/uc?id=1QRT2rbR6fPHvwyzXyNkKfrtv0r_MBuzt)

## 기대 효과
- **몰입형 디지털 스토리텔링 경험**: 사진과 AI가 결합된 새로운 콘텐츠 제공
- **사용자 창의성 극대화**: 즉각적 피드백·재생성을 통한 풍부한 창작 과정
- **지속 가능한 생태계 조성**: 교육, 마케팅, 예술 등 다방면 활용·확장 가능

---

## 결론 및 향후 과제

본 프로젝트는 개인화된 인터랙티브 스토리텔링의 새로운 패러다임을 제안합니다.  
향후 모델 고도화, UI/UX 개선, 교육·문화 예술 분야 협업, 수익화 모델 검토 등을 통해  
한 단계 더 발전시켜나갈 예정입니다.
